In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import lightning as L

In [2]:
# create vocabulary
token_to_id = {
    "who": 0,
    "is": 1,
    "charley": 2,
    "an": 3,
    "awesome": 4,
    "guy": 5,
    "<EOS>": 6,
}

# create reverse
id_to_token = dict(map(reversed, token_to_id.items()))

In [3]:
# create inputs
inputs = torch.tensor([
    [token_to_id["who"],
     token_to_id["is"],
     token_to_id["charley"],
     token_to_id["<EOS>"],
     token_to_id["an"],
     token_to_id["awesome"],
     token_to_id["guy"]],

    [token_to_id["charley"],
     token_to_id["is"],
     token_to_id["who"],
     token_to_id["<EOS>"],
     token_to_id["an"],
     token_to_id["awesome"],
     token_to_id["guy"]]
])

# create labels
labels = torch.tensor([
    [token_to_id["is"],
     token_to_id["charley"],
     token_to_id["<EOS>"],
     token_to_id["an"],
     token_to_id["awesome"],
     token_to_id["guy"],
     token_to_id["<EOS>"]],

    [token_to_id["is"],
     token_to_id["who"],
     token_to_id["<EOS>"],
     token_to_id["an"],
     token_to_id["awesome"],
     token_to_id["guy"],
     token_to_id["<EOS>"]]
])

# create dataset from inputs and labels
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [4]:
# position encoding
class PositionEncoding(nn.Module):

    def __init__(self, d_model=2, max_len=6):

        super().__init__()

        pe = torch.zeros(max_len, d_model)

        position = torch.arange(start=0, end=max_len, step=1).float().unsqueeze(1)
        embedding_index = torch.arange(start=0, end=d_model, step=2).float()

        div_term = 1/torch.tensor(10000.0) ** (embedding_index / d_model)

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe)

    def forward(self, word_embeddings):

        return word_embeddings + self.pe[:word_embeddings.size(0), :]

In [5]:
# multi headed attention
class Attention(nn.Module):

    def __init__(self, d_model=2):

        super().__init__()

        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)

        self.row_dim = 0
        self.col_dim = 1

    def forward(self, encodings_for_q, encodings_for_k, encodings_for_v, mask=None):

        q = self.W_q(encodings_for_q)
        k = self.W_k(encodings_for_k)
        v = self.W_v(encodings_for_v)

        sims = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim))

        scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)

        if mask is not None:
            scaled_sims = scaled_sims.masked_fill(mask=mask, value=-1e9)

        attention_percents = F.softmax(scaled_sims, dim=self.col_dim)

        attention_scores = torch.matmul(attention_percents, v)

        return attention_scores



In [6]:
# now create transformer
class DecoderOnlyTransformer(L.LightningModule):

    def __init__(self, num_tokens=7, d_model=2, max_len=8):

        super().__init__()

        self.we = nn.Embedding(num_embeddings=num_tokens,
                               embedding_dim=d_model)
        
        self.pe = PositionEncoding(d_model=d_model,
                                   max_len=max_len)
        
        self.self_attention = Attention(d_model=d_model)

        self.fc_layer = nn.Linear(in_features=d_model, out_features=num_tokens)

        self.loss = nn.CrossEntropyLoss()

    def forward(self, token_ids):

        word_embeddings = self.we(token_ids)
        position_encoded = self.pe(word_embeddings)

        mask = torch.tril(torch.ones((token_ids.size(dim=0), token_ids.size(dim=0))))
        mask = mask == 0

        mask = mask.to(device="cuda")

        self_attention_values = self.self_attention(position_encoded,
                                                    position_encoded,
                                                    position_encoded,
                                                    mask=mask)
        
        residual_connection_values = position_encoded + self_attention_values

        fc_layer_output = self.fc_layer(residual_connection_values)

        return fc_layer_output
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.1)
    
    def training_step(self, batch, batch_idx):
        input_tokens, labels = batch
        output = self.forward(input_tokens[0])
        loss = self.loss(output, labels[0])

        return loss
    
     

In [7]:
model = DecoderOnlyTransformer(num_tokens=len(token_to_id), d_model=2, max_len=8)
model = model.to("cuda")
model_input = torch.tensor(
    [token_to_id["who"],
     token_to_id["is"],
     token_to_id["charley"],
     token_to_id["<EOS>"]])
model_input = model_input.to("cuda")
input_length = model_input.size(dim=0)

predictions = model(model_input)
predicted_id = torch.tensor([torch.argmax(predictions[-1, :])])
predicted_ids = predicted_id

max_length = 8
for i in range(input_length, max_length):
    predicted_id = predicted_id.to("cuda")
    
    if (predicted_id == token_to_id["<EOS>"]):
        break

    model_input = torch.cat((model_input, predicted_id))

    predictions = model(model_input)
    predicted_id = torch.tensor([torch.argmax(predictions[-1, :])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))

print("Predicted Tokens:\n")
for id in predicted_ids:
    print("\t", id_to_token[id.item()])

Predicted Tokens:

	 awesome
	 charley
	 <EOS>


In [8]:
trainer = L.Trainer(max_epochs=30)
trainer.fit(model, train_dataloaders=dataloader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/charley/programming/gpt-transformer-pytorch/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/home/charley/programming/gpt-transformer-pytorch/venv/lib/python3.10/site-packages/torch/__init__.py:1551: Use

Epoch 29: 100%|██████████| 2/2 [00:00<00:00, 149.29it/s, v_num=3]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 2/2 [00:00<00:00, 102.33it/s, v_num=3]


In [10]:
model =model.to("cuda")

model_input = torch.tensor(
    [token_to_id["who"],
     token_to_id["is"],
     token_to_id["charley"],
     token_to_id["<EOS>"]])
model_input = model_input.to("cuda")
input_length = model_input.size(dim=0)

predictions = model(model_input)
predicted_id = torch.tensor([torch.argmax(predictions[-1, :])])
predicted_ids = predicted_id

max_length = 8
for i in range(input_length, max_length):
    predicted_id = predicted_id.to("cuda")
    
    if (predicted_id == token_to_id["<EOS>"]):
        break

    model_input = torch.cat((model_input, predicted_id))

    predictions = model(model_input)
    predicted_id = torch.tensor([torch.argmax(predictions[-1, :])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))

print("Predicted Tokens:\n")
for id in predicted_ids:
    print("\t", id_to_token[id.item()])

Predicted Tokens:

	 an
	 awesome
	 guy
	 <EOS>


In [11]:
model_input = torch.tensor(
    [token_to_id["charley"],
     token_to_id["is"],
     token_to_id["who"],
     token_to_id["<EOS>"]])
model_input = model_input.to("cuda")
input_length = model_input.size(dim=0)

predictions = model(model_input)
predicted_id = torch.tensor([torch.argmax(predictions[-1, :])])
predicted_ids = predicted_id

max_length = 8
for i in range(input_length, max_length):
    predicted_id = predicted_id.to("cuda")
    
    if (predicted_id == token_to_id["<EOS>"]):
        break

    model_input = torch.cat((model_input, predicted_id))

    predictions = model(model_input)
    predicted_id = torch.tensor([torch.argmax(predictions[-1, :])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))

print("Predicted Tokens:\n")
for id in predicted_ids:
    print("\t", id_to_token[id.item()])

Predicted Tokens:

	 an
	 awesome
	 guy
	 <EOS>
